In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import keras
from keras import layers
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import save_model, load_model

2022-05-05 00:49:19.318495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :
2022-05-05 00:49:19.318554: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
counts_aorta= pd.read_csv("data/Aorta-counts.csv")
counts_aorta.set_index("Unnamed: 0", inplace=True)
counts_aorta = counts_aorta.T
counts_aorta.head()

Unnamed: 0,0610005C13Rik,0610007C21Rik,0610007L01Rik,0610007N19Rik,0610007P08Rik,0610007P14Rik,0610007P22Rik,0610008F07Rik,0610009B14Rik,0610009B22Rik,...,Zxdb,Zxdc,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3,a,l7Rn6,zsGreen_transgene
A21.MAA000594.3_8_M.1.1,0,406,714,0,358,0,0,0,0,242,...,0,0,0,0,1150,0,0,0,345,0
D1.MAA000594.3_8_M.1.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,142,0,0,0,0,0
F8.MAA000594.3_8_M.1.1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H11.MAA000594.3_8_M.1.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,281,0,0,0,0
N15.MAA000594.3_8_M.1.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
counts_kidney= pd.read_csv("data/Kidney-counts.csv")
counts_kidney.head()

,Unnamed: 0,A14.MAA000545.3_8_M.1.1,E1.MAA000545.3_8_M.1.1,M4.MAA000545.3_8_M.1.1,O21.MAA000545.3_8_M.1.1,P4.MAA000545.3_8_M.1.1,A16.MAA000545.3_8_M.1.1,E17.MAA000545.3_8_M.1.1,M5.MAA000545.3_8_M.1.1,A21.MAA000545.3_8_M.1.1,...,N4.B002775.3_39_F.1.1,P2.B002775.3_39_F.1.1,P3.B002775.3_39_F.1.1,N5.B002775.3_39_F.1.1,N6.B002775.3_39_F.1.1,P4.B002775.3_39_F.1.1,N7.B002775.3_39_F.1.1,P6.B002775.3_39_F.1.1,N8.B002775.3_39_F.1.1,P7.B002775.3_39_F.1.1
0,0610005C13Rik,0,0,0,0,318,0,150,0,9,...,18,0,0,0,0,0,0,0,0,0
1,0610007C21Rik,1,0,0,0,191,1,45,10,96,...,0,0,0,0,0,0,333,0,0,0
2,0610007L01Rik,0,0,0,0,0,0,0,0,0,...,0,0,339,0,0,0,0,0,0,0
3,0610007N19Rik,0,0,0,0,0,1,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0610007P08Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
counts_liver= pd.read_csv("data/Liver-counts.csv")
counts_liver.head()

,Unnamed: 0,F18.MAA000377.3_9_M.1.1,J20.MAA000377.3_9_M.1.1,F19.MAA000377.3_9_M.1.1,J21.MAA000377.3_9_M.1.1,F20.MAA000377.3_9_M.1.1,J22.MAA000377.3_9_M.1.1,G11.MAA000377.3_9_M.1.1,K9.MAA000377.3_9_M.1.1,G13.MAA000377.3_9_M.1.1,...,A2.MAA100042.3_9_M.1.1,C4.MAA100042.3_9_M.1.1,A5.MAA100042.3_9_M.1.1,C6.MAA100042.3_9_M.1.1,A6.MAA100042.3_9_M.1.1,C7.MAA100042.3_9_M.1.1,A9.MAA100042.3_9_M.1.1,C9.MAA100042.3_9_M.1.1,A10.MAA100042.3_9_M.1.1,C10.MAA100042.3_9_M.1.1
0,0610005C13Rik,0,0,0,0,0,0,0,0,168,...,0,99,0,292,0,310,392,220,0,0
1,0610007C21Rik,0,0,0,0,2,58,1,0,0,...,0,753,0,630,1,768,527,129,0,0
2,0610007L01Rik,23,0,0,0,0,0,1,5,0,...,0,0,0,35,0,90,0,23,0,0
3,0610007N19Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0610007P08Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
counts_lung= pd.read_csv("data/Lung-counts.csv")
counts_lung.head()

,Unnamed: 0,A7.MAA000526.3_9_M.1.1,B12.MAA000526.3_9_M.1.1,D4.MAA000526.3_9_M.1.1,E11.MAA000526.3_9_M.1.1,G6.MAA000526.3_9_M.1.1,H14.MAA000526.3_9_M.1.1,J4.MAA000526.3_9_M.1.1,K21.MAA000526.3_9_M.1.1,M10.MAA000526.3_9_M.1.1,...,N9.D043522.3_39_F.1.1,P15.D043522.3_39_F.1.1,J22.D043522.3_39_F.1.1,L20.D043522.3_39_F.1.1,N10.D043522.3_39_F.1.1,P17.D043522.3_39_F.1.1,N11.D043522.3_39_F.1.1,P20.D043522.3_39_F.1.1,K1.D043522.3_39_F.1.1,L21.D043522.3_39_F.1.1
0,0610005C13Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0610007C21Rik,310,200,0,485,546,408,266,0,4,...,0,871,0,0,0,110,0,217,60,0
2,0610007L01Rik,0,45,0,1,0,126,147,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0610007N19Rik,0,119,0,81,0,37,2,1,77,...,0,0,0,0,0,0,0,0,2,0
4,0610007P08Rik,0,0,0,117,0,41,0,0,0,...,0,0,0,0,0,0,0,0,0,0
